In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import  StandardScaler
from wandb import sklearn

# Data understanding

## Gathering data

In [2]:
url = "https://github.com/dicodingacademy/dicoding_dataset/blob/main/employee/employee_data.csv?raw=true"
df = pd.read_csv(url)
df.head()

,EmployeeId,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1,38,NaN,Travel_Frequently,1444,Human Resources,1,4,Other,1,...,2,80,1,7,2,3,6,2,1,2
1,2,37,1.0,Travel_Rarely,1141,Research & Development,11,2,Medical,1,...,1,80,0,15,2,1,1,0,0,0
2,3,51,1.0,Travel_Rarely,1323,Research & Development,4,4,Life Sciences,1,...,3,80,3,18,2,4,10,0,2,7
3,4,42,0.0,Travel_Frequently,555,Sales,26,3,Marketing,1,...,4,80,1,23,2,4,20,4,4,8
4,5,40,NaN,Travel_Rarely,1194,Research & Development,2,4,Medical,1,...,2,80,3,20,2,3,5,3,0,2


#### Insight data

is got 34 columns as follows:

* **EmployeeId** - Employee Identifier
* **Attrition** - Did the employee attrition? (0=no, 1=yes)
* **Age** - Age of the employee
* **BusinessTravel** - Travel commitments for the job
* **DailyRate** - Daily salary
* **Department** - Employee Department
* **DistanceFromHome** - Distance from work to home (in km)
* **Education** - 1-Below College, 2-College, 3-Bachelor, 4-Master,5-Doctor
* **EducationField** - Field of Education
* **EnvironmentSatisfaction** - 1-Low, 2-Medium, 3-High, 4-Very High
* **Gender** - Employee's gender
* **HourlyRate** - Hourly salary
* **JobInvolvement** - 1-Low, 2-Medium, 3-High, 4-Very High
* **JobLevel** - Level of job (1 to 5)
* **JobRole** - Job Roles
* **JobSatisfaction** - 1-Low, 2-Medium, 3-High, 4-Very High
* **MaritalStatus** - Marital Status
* **MonthlyIncome** - Monthly salary
* **MonthlyRate** - Mounthly rate
* **NumCompaniesWorked** - Number of companies worked at
* **Over18** - Over 18 years of age?
* **OverTime** - Overtime?
* **PercentSalaryHike** - The percentage increase in salary last year
* **PerformanceRating** - 1-Low, 2-Good, 3-Excellent, 4-Outstanding
* **RelationshipSatisfaction** - 1-Low, 2-Medium, 3-High, 4-Very High
* **StandardHours** - Standard Hours
* **StockOptionLevel** - Stock Option Level
* **TotalWorkingYears** - Total years worked
* **TrainingTimesLastYear** - Number of training attended last year
* **WorkLifeBalance** - 1-Low, 2-Good, 3-Excellent, 4-Outstanding
* **YearsAtCompany** - Years at Company
* **YearsInCurrentRole** - Years in the current role
* **YearsSinceLastPromotion** - Years since the last promotion
* **YearsWithCurrManager** - Years with the current manager

## Assessing data

### Check many data

In [3]:
print(f"many data:{len(df)}")

many data:1470


### Check type data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmployeeId                1470 non-null   int64  
 1   Age                       1470 non-null   int64  
 2   Attrition                 1058 non-null   float64
 3   BusinessTravel            1470 non-null   object 
 4   DailyRate                 1470 non-null   int64  
 5   Department                1470 non-null   object 
 6   DistanceFromHome          1470 non-null   int64  
 7   Education                 1470 non-null   int64  
 8   EducationField            1470 non-null   object 
 9   EmployeeCount             1470 non-null   int64  
 10  EnvironmentSatisfaction   1470 non-null   int64  
 11  Gender                    1470 non-null   object 
 12  HourlyRate                1470 non-null   int64  
 13  JobInvolvement            1470 non-null   int64  
 14  JobLevel

#### Insight
Is got anomalie data type which must isn't happened
* **EmployeeID** - string


### Check missing value

In [5]:
df.isnull().sum()

EmployeeId                    0
Age                           0
Attrition                   412
BusinessTravel                0
DailyRate                     0
Department                    0
DistanceFromHome              0
Education                     0
EducationField                0
EmployeeCount                 0
EnvironmentSatisfaction       0
Gender                        0
HourlyRate                    0
JobInvolvement                0
JobLevel                      0
JobRole                       0
JobSatisfaction               0
MaritalStatus                 0
MonthlyIncome                 0
MonthlyRate                   0
NumCompaniesWorked            0
Over18                        0
OverTime                      0
PercentSalaryHike             0
PerformanceRating             0
RelationshipSatisfaction      0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
WorkLifeBalance               0
YearsAtC

#### Insight
Obtained 412 null data in the Attrition column

### Check duplicate data

In [6]:
df.duplicated().sum()

0

#### Insight
Is not got duplicate data

### Check value unique

In [7]:
df["BusinessTravel"].value_counts()

BusinessTravel
Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: count, dtype: int64

In [8]:
df["Department"].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [9]:
df["EducationField"].value_counts()

EducationField
Life Sciences       606
Medical             464
Marketing           159
Technical Degree    132
Other                82
Human Resources      27
Name: count, dtype: int64

In [10]:
df["Gender"].value_counts()

Gender
Male      882
Female    588
Name: count, dtype: int64

In [11]:
df["JobRole"].value_counts()

JobRole
Sales Executive              326
Research Scientist           292
Laboratory Technician        259
Manufacturing Director       145
Healthcare Representative    131
Manager                      102
Sales Representative          83
Research Director             80
Human Resources               52
Name: count, dtype: int64

In [12]:
df["MaritalStatus"].value_counts()

MaritalStatus
Married     673
Single      470
Divorced    327
Name: count, dtype: int64

In [13]:
df["Over18"].value_counts()

Over18
Y    1470
Name: count, dtype: int64

In [14]:
df["OverTime"].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

#### Insight
all columns categorical will be changed into numerical
* **BusinessTravel**
* **Departement**
* **EducationField**
* **Gender**
* **JobRole**
* **MaritalStatus**
* **Over18**
* **OverTime**

## Cleaning data

### To change type date
change type data to column which has been done assessing data

In [15]:
df.rename(columns={"EmployeeId" : "EmployeId"}, inplace=True)
df["EmployeId"] = df["EmployeId"].astype(str)


In [16]:
df.head()

,EmployeId,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1,38,NaN,Travel_Frequently,1444,Human Resources,1,4,Other,1,...,2,80,1,7,2,3,6,2,1,2
1,2,37,1.0,Travel_Rarely,1141,Research & Development,11,2,Medical,1,...,1,80,0,15,2,1,1,0,0,0
2,3,51,1.0,Travel_Rarely,1323,Research & Development,4,4,Life Sciences,1,...,3,80,3,18,2,4,10,0,2,7
3,4,42,0.0,Travel_Frequently,555,Sales,26,3,Marketing,1,...,4,80,1,23,2,4,20,4,4,8
4,5,40,NaN,Travel_Rarely,1194,Research & Development,2,4,Medical,1,...,2,80,3,20,2,3,5,3,0,2


### To change columns categorical to numeric
changing columns categorical faceilitate for getting feature

In [17]:
df_pre = df.copy()
df_pre.drop("EmployeId", axis=1, inplace=True)

for column in df_pre.columns:
    if df_pre[column].dtype == "object":
        df_pre[column] = df_pre[column].astype("category")
        df_pre[column] = df_pre[column].cat.codes

In [18]:
df_pre.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,38,NaN,1,1444,0,1,4,4,1,4,...,2,80,1,7,2,3,6,2,1,2
1,37,1.0,2,1141,1,11,2,3,1,1,...,1,80,0,15,2,1,1,0,0,0
2,51,1.0,2,1323,1,4,4,1,1,1,...,3,80,3,18,2,4,10,0,2,7
3,42,0.0,1,555,2,26,3,2,1,3,...,4,80,1,23,2,4,20,4,4,8
4,40,NaN,2,1194,1,2,4,3,1,3,...,2,80,3,20,2,3,5,3,0,2


In [19]:
df_pre["BusinessTravel"].value_counts()

BusinessTravel
2    1043
1     277
0     150
Name: count, dtype: int64

In [20]:
df_pre["Department"].value_counts()

Department
1    961
2    446
0     63
Name: count, dtype: int64

In [21]:
df_pre["EducationField"].value_counts()

EducationField
1    606
3    464
2    159
5    132
4     82
0     27
Name: count, dtype: int64

In [22]:
df_pre["Gender"].value_counts()

Gender
1    882
0    588
Name: count, dtype: int64

In [23]:
df_pre["JobRole"].value_counts()

JobRole
7    326
6    292
2    259
4    145
0    131
3    102
8     83
5     80
1     52
Name: count, dtype: int64

In [24]:
df_pre["MaritalStatus"].value_counts()

MaritalStatus
1    673
2    470
0    327
Name: count, dtype: int64

In [25]:
df_pre["Over18"].value_counts()

Over18
0    1470
Name: count, dtype: int64

In [26]:
df_pre["OverTime"].value_counts()

OverTime
0    1054
1     416
Name: count, dtype: int64

#### Insight
explaining columns which are done changing:
1. BusinessTravel - 0-Non-Travel, 1-Travel_Frequently, 2-Travel_Rarely
2. Department - 0-Human Resources, 1-Research & Development, 2-sales
3. EducationField - 0-Human Resources, 1-Life Sciences, 2-Marketing, 3-Medical, 4-Other, 5-Technical Degree
4. Gender - 0-Female, 1-Male
5. JobRole - 0-Healthcare Representative, 1-Human Resource, 2-Laboratory Technician, 3-Manager, 4-Manufacuring Director,5-Research Director, 6-Research Scientist, 7-Sales Excutive, 8-Sales Representative
6. MaritalStatus - 0-Divorced, 1-Married, 2-single
7. Over18 - 0-Y, 1-N
8. OverTime - 0-No, 1-Yes



### To handle Missing values
because missing values is big which is not possible for dropping data, but there are many method for handling missing values which we will use one of method for it

In [27]:
not_missing = df_pre[df_pre["Attrition"].notnull()]
missing = df_pre[df_pre["Attrition"].isnull()]

# drop columns over because we don't need
not_missing = not_missing.drop(columns=["Over18"], axis=1)
missing = missing.drop(columns=["Over18"], axis=1)

In [28]:
not_missing["Attrition"].value_counts()

Attrition
0.0    879
1.0    179
Name: count, dtype: int64

In [29]:
attrition_zero_df = not_missing[not_missing["Attrition"] == 0].sample(n=179, random_state=42)
attrition_one_df = not_missing[not_missing["Attrition"] == 1].sample(n=179, random_state=42)

df_missing = pd.concat([attrition_zero_df, attrition_one_df])

In [30]:
df_missing["Attrition"].value_counts()

Attrition
0.0    179
1.0    179
Name: count, dtype: int64

In [38]:
# initialize
scaler = StandardScaler()
x = df_missing.drop("Attrition", axis=1)
y = df_missing["Attrition"]

# Standarization feature
X = scaler.fit_transform(x)

model_random = RandomForestClassifier()
model_random.fit(X, y)
print(f"score accuracy:{model_random.score(X, y) * 100:.2f}%")


score accuracy:100.00%


In [35]:
# predict missing value
predicted = model_random.predict(missing.drop("Attrition", axis=1))

# fill in missing value
df_pre.loc[df_pre['Attrition'].isnull(), 'Attrition'] = predicted

/Users/acqmallatief/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [36]:
df_pre["Attrition"].value_counts()

Attrition
0.0    941
1.0    529
Name: count, dtype: int64

In [37]:
df_pre.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

# Data preprocessing

# Modeling

# Evalution